<a href="https://colab.research.google.com/github/BouzCS/Text-Generator-w-BERT/blob/main/Text_Generator_from_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')


def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess


In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

model_name = 'cointegrated/rubert-tiny'
model_score = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')


def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess


In [6]:
import numpy as np

X=['I','went','school']
i=0
f=len(X)
global_sentences = []

while i<f:
  X.insert(i+2,"___")
  sentence=" ".join(X)
  sentence = sentence.replace("___","<mask>")
  predicted_blanks = get_prediction(sentence)
  predicted_blanks=predicted_blanks.split(" ")[1:]
  sentence=(sentence).split(" ")
  sentence=np.array(sentence)
  index, = np.where(sentence == "<mask>")
  for j in range(len(predicted_blanks)):
    sentence[index[j]]=predicted_blanks[j]
    final=" ".join(list(sentence))
    
  global_sentences.append(final)
  i=i+2


In [7]:
X=['I','went','school']
i=0
f=len(X)
while i+1<f:
  X.insert(f-1,"___")
  sentence=" ".join(X)

  sentence = sentence.replace("___","<mask>")
  predicted_blanks = get_prediction(sentence)
  predicted_blanks=predicted_blanks.split(" ")[1:]
  
  sentence=(sentence).split(" ")
  sentence=np.array(sentence)
  index, = np.where(sentence == "<mask>")

  for j in range(len(predicted_blanks)):
    sentence[index[j]]=predicted_blanks[j]
    final=" ".join(list(sentence))

  global_sentences.append(final)
  f=f-1



In [8]:
global_sentences=list(set(global_sentences))

In [11]:
global_sentences

['I never went to school', 'I went to school', 'I went to school .']

In [18]:
global_scores=[]
for text in global_sentences:
  s=score(sentence=text, model=model, tokenizer=tokenizer)
  global_scores.append(s)

In [19]:
global_scores
max_index = global_scores.index(max(global_scores))

In [20]:
global_sentences[max_index]

'I went to school .'

In [33]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')


def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess


model_name = 'cointegrated/rubert-tiny'
model_score = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt')
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')


def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess

def GeneratorText(words):
  X=words
  Z=words.copy()
  i=0
  f=len(X)
  global_sentences = []

#go backward in the list

  while i<f:
    X.insert(i+2,"___")
    sentence=" ".join(X)
    sentence = sentence.replace("___","<mask>")
    predicted_blanks = get_prediction(sentence)
    predicted_blanks=predicted_blanks.split(" ")[1:]
    sentence=(sentence).split(" ")
    sentence=np.array(sentence)
    index, = np.where(sentence == "<mask>")
    for j in range(len(predicted_blanks)):
      sentence[index[j]]=predicted_blanks[j]
      final=" ".join(list(sentence))
      
    global_sentences.append(final)
    i=i+2
  
#go forward in the list
  i=0
  f=len(Z)
  while i+1<f:
    Z.insert(f-1,"___")
    sentence=" ".join(Z)


    sentence = sentence.replace("___","<mask>")
    predicted_blanks = get_prediction(sentence)
    predicted_blanks=predicted_blanks.split(" ")[1:]
    
    sentence=(sentence).split(" ")
    sentence=np.array(sentence)
    index, = np.where(sentence == "<mask>")

    for j in range(len(predicted_blanks)):
      sentence[index[j]]=predicted_blanks[j]
      final=" ".join(list(sentence))

    global_sentences.append(final)
    f=f-1

#remove duplicates

  global_sentences=list(set(global_sentences))
  global_scores=[]

#get scores of generated texts

  for text in global_sentences:
    s=score(sentence=text, model=model, tokenizer=tokenizer)
    global_scores.append(s)

#get max score of the predicted generated sentences

  max_index = global_scores.index(max(global_scores))

  return global_sentences[max_index]

print(GeneratorText(['I','went','school']))


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


I went to school .
